In [33]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torchvision import models
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [34]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [35]:
#torch.manual_seed(0)
batch_size = 256
test_dataset = torchvision.datasets.ImageFolder(root='test', transform=transform)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [36]:
resnet = models.resnet152(weights=True)
for param in resnet.parameters():
    param.requires_grad = False

num_classes = 8
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Sequential( 
    nn.Linear(num_ftrs, 1024),
    nn.ReLU(),
    nn.BatchNorm1d(1024),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.BatchNorm1d(256),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.BatchNorm1d(128),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.BatchNorm1d(64),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.BatchNorm1d(32),
    nn.Linear(32, num_classes)
)

resnet.add_module('softmax', nn.Softmax(dim=1))

/home/da109073/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [37]:
model_path = 'model.pth'
state_dict = torch.load(model_path)
resnet.load_state_dict(state_dict)
resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [38]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [39]:
predictions = []
targets = []

for inputs, labels in testloader:
    outputs = resnet(inputs)
    _, predicted = torch.max(outputs, 1)
    predictions.extend(predicted.tolist())
    targets.extend(labels.tolist())

predictions = torch.tensor(predictions)
targets = torch.tensor(targets)
accuracy = torch.mean((predictions == targets).float()).item() * 100
print(f"Accuracy: {accuracy:.2f}%")

/home/da109073/anaconda3/lib/python3.11/site-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Accuracy: 90.00%
